In [2]:
import json
import os
import pandas as pd

from dotenv import load_dotenv
from openrouteservice import client, places

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
output_dir = '../data/curated/'

## set variables
load_dotenv()
tokens = []
for i in range(1,17): 
    token = os.environ.get(f'token_{i}')
    tokens.append(token)
category_ids = [587, 588, 604, 607]
buffer_size = 2000   # between 1-2000
ptv_dict = {}

In [4]:
## read apartment data
property_data = json.load(open( f"{output_dir}properties_processed.json" ))

In [229]:
token = tokens[0]

## set x and y
x = len(ptv_dict)
y = len(ptv_dict)+100

for index in range(x,y):
    if index > len(property_data['Coordinates'].keys()):
        break
    backwards = list(map(float,property_data['Coordinates'][str(index)][1:-1].split(',')))
    coords = [backwards[1], backwards[0]]

    ## query code
    ors = client.Client(key=token)
    query = {'request': 'pois',
            'geojson': {'type':'Point','coordinates':coords},
            'buffer': buffer_size,
            'filter_category_ids': category_ids,
            'sortby':'distance',
            'limit': 4}
    features = ors.places(**query)['features']

    ## distances - direct meters
    distances = []
    for poi in features:
        distances.append(poi['properties']['distance'])
        
    ## add information
    ptv_dict[str(index)] = distances

KeyError: '7526'

In [230]:
print(len(ptv_dict))

7526


In [231]:
property_data['PTV'] = ptv_dict

## write json
property_data['PTV'] = ptv_dict
json.dump(property_data, open(f"{output_dir}num_ptv.json", 'w'))

# write csv
data = pd.read_json(f"{output_dir}num_ptv.json")
data.to_csv(f"{output_dir}num_ptv.csv")